<a href="https://colab.research.google.com/github/minsaee/ai_chat_python/blob/master/405_pandas_%EC%98%81%ED%99%94%EC%B6%94%EC%B2%9C%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ai_chat_python

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ai_chat_python


In [7]:
%%writefile movie_system.py

import numpy as np
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# 파일로딩
def data_load():
  movies = pd.read_csv('./source/m1/movies.dat', delimiter = '::',header =None, engine ='python', encoding='ISO-8859-1',
                       names=['MovieID','Title','Genres'])
  users = pd.read_csv('./source/m1/users.dat', sep ='::' , engine ='python' ,header = None,
                      names=['UserID','Gender','Age','Occupation','Zip-code'])
  ratings = pd.read_csv('./source/m1/ratings.dat', sep = '::', engine ='python' ,header = None,
                        names=['UserID','MovieID','Rating','Timestamp'])
  return movies, users, ratings

# merge
def data_merge(movies, users, ratings):
  data = ratings.merge(users).merge(movies)
  recommendation_data = data[['UserID','MovieID','Rating']]
  return recommendation_data

# pivot, corr
def data_pivot_corr(recommendation_data):
  recommendation_pivot = recommendation_data.pivot(index ='UserID',columns='MovieID',values='Rating')
  recommendation_pivot.fillna(0, inplace =True)
  return recommendation_pivot

def nearest_user(small_test_corr, user_id, n):
  return small_test_corr.loc[user_id].sort_values(ascending = False)[1: n+1]

def movie_seen(user_id):
  return recommendation_pivot.loc[user_id][recommendation_pivot.loc[user_id]>0]

def recommend_movie(small_test_corr, movies, user_id, n):
  small_test_corr = recommendation_pivot.T.iloc[:500,:500].corr() # 500개 제한
  user_list = nearest_user(small_test_corr, user_id, n).index
  user_mv_list = recommendation_data[(recommendation_data.UserID.isin(user_list))&(recommendation_data.Rating==5)]
  user7_mv_list = movie_seen(user_id)
  unseen_list = set(user_mv_list['MovieID']) - set(user7_mv_list.index)
  return movies[movies['MovieID'].isin(unseen_list)].reset_index(drop=True)

if __name__ == '__main__':
  movies, users, ratings = data_load()
  recommendation_data = data_merge(movies, users, ratings)
  recommendation_pivot = data_pivot_corr(recommendation_data)
  userid = int(input('UserID 입력:'))
  movie_receive = recommend_movie(recommendation_pivot, movies, userid, 2)
  print(len(movie_receive))
  print(movie_receive)

Overwriting movie_system.py


In [8]:
!pwd

/content/drive/MyDrive/ai_chat_python


In [10]:
!python movie_system.py

UserID 입력:4
15
    MovieID                                      Title                           Genres
0       733                           Rock, The (1996)        Action|Adventure|Thriller
1      1079                Fish Called Wanda, A (1988)                           Comedy
2      1136     Monty Python and the Holy Grail (1974)                           Comedy
3      1197                 Princess Bride, The (1987)  Action|Adventure|Comedy|Romance
4      1259                         Stand by Me (1986)           Adventure|Comedy|Drama
5      1266                          Unforgiven (1992)                          Western
6      1286                   Somewhere in Time (1980)                    Drama|Romance
7      1304  Butch Cassidy and the Sundance Kid (1969)            Action|Comedy|Western
8      1378                          Young Guns (1988)            Action|Comedy|Western
9      1615                           Edge, The (1997)               Adventure|Thriller
10     2167      